In [1]:
import pandas as pd
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df_requests = pd.read_excel('FRC Data1.xlsx', sheet_name='FRC Requests')
# df_people = pd.read_excel('FRC Data1.xlsx', sheet_name='Requestor')

In [35]:
df_requests['Category'].value_counts()

Process    3727
Systems    1859
Forms      1458
Rate       1140
Other       457
Name: Category, dtype: int64

In [24]:
df_requests['Key Word Roll-Up'].value_counts()

Rate Change                 556
Form Review                 262
Endorsement Print           250
Premium Discrepancy         218
CAT                         194
Endorsement Process         186
Experience Modification     181
Billing                     166
System Issue                165
Policy Setup                157
Rating Error                152
BOR                         135
Policy Issuance             130
Rating Process              125
Officer Exclusion           123
Loss Analysis               123
LOB Assistance              121
Premium Issue               115
RE Factor                   114
Technical Error             113
Filing                      110
Audit                       105
User Access                 103
Classification              100
Technical Rating             94
Form Missing                 90
Policy Print                 89
Cancel Rewrite               87
Composite                    85
Endorsement Correction       85
                           ... 
CNA Emer

In [5]:
df_requests.head()

,ID,Submission Date,Resolution Date,FRC Owner,Request,Requestor,Request Description,Status,Urgent?,Reopen?,...,Resource(s) Used / Referred To,Incident / PME / Project # (Where Appl.),Created,Created By,Modified,Last Modified By,Submission Month,res date,Item Type,Path
0,13960,2018-10-01,NaT,"Bush, Andrew",return premium endorsement was booked in Rapid,"Lal,Pushpanjali","\n​\nGood Morning All,\n \nAs I mentioned belo...",Open,No,No,...,NaN,NaN,2018-10-01 10:44:16,"Munsell,Julia E.",2018-10-01 10:44:16,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST
1,13959,2018-10-01,NaT,"Reel, Joshua",CRIS training ?,"Albright,Kevin T.",\n​Please advise the path for an employee to r...,Open,No,No,...,NaN,NaN,2018-10-01 10:33:36,"Munsell,Julia E.",2018-10-01 10:33:36,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST
2,13958,2018-10-01,NaT,"Brooks, Jayson",reference connect,"Amato,Marisol",\n​\nHello\n \nI need to get a couple of publi...,Open,No,No,...,NaN,NaN,2018-10-01 10:32:12,"Munsell,Julia E.",2018-10-01 10:32:12,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST
3,13957,2018-10-01,NaT,"Bush, Andrew",Tech Rating,"Miletich,Denise",\n​Tech RAting error,Pending Follow-Up Outside FRC,No,No,...,TAP IT,INC0692683,2018-10-01 10:29:46,"Bush,Andrew",2018-10-01 10:29:46,"Bush,Andrew",201810,-,Item,ccm/FRC/Lists/TEST
4,13956,2018-10-01,NaT,"Ewing, Jamie",Submission Documents in EPC,"Howard,Anne",\n​\nI'm not able to open new business submiss...,Open,No,No,...,NaN,NaN,2018-10-01 09:40:43,"Munsell,Julia E.",2018-10-01 09:40:43,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST


In [3]:
key_word_list = ['Rate Change',
'Form Review',
'Endorsement Print',
'Premium Discrepancy',
'CAT',
'Endorsement Process',
'Experience Modification',
'Billing',
'System Issue',
'Policy Setup',
'Rating Error',
'BOR',
'Policy Issuance',
'Rating Process',
'Officer Exclusion',
'Loss Analysis',
'LOB Assistance'
'Premium Issue']
condensed_list = []
for row in df_requests['Key Word Roll-Up']:
    if row in key_word_list:
        condensed_list.append(row)
    else: 
        condensed_list.append('Other')

In [5]:
len(df_requests)

8642

In [ ]:
len()

In [6]:
df_requests['key_words'] = condensed_list

In [7]:
df_requests.head()

,ID,Submission Date,Resolution Date,FRC Owner,Request,Requestor,Request Description,Status,Urgent?,Reopen?,...,Incident / PME / Project # (Where Appl.),Created,Created By,Modified,Last Modified By,Submission Month,res date,Item Type,Path,key_words
0,13960,2018-10-01,NaT,"Bush, Andrew",return premium endorsement was booked in Rapid,"Lal,Pushpanjali","\n​\nGood Morning All,\n \nAs I mentioned belo...",Open,No,No,...,NaN,2018-10-01 10:44:16,"Munsell,Julia E.",2018-10-01 10:44:16,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST,Endorsement Process
1,13959,2018-10-01,NaT,"Reel, Joshua",CRIS training ?,"Albright,Kevin T.",\n​Please advise the path for an employee to r...,Open,No,No,...,NaN,2018-10-01 10:33:36,"Munsell,Julia E.",2018-10-01 10:33:36,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST,Other
2,13958,2018-10-01,NaT,"Brooks, Jayson",reference connect,"Amato,Marisol",\n​\nHello\n \nI need to get a couple of publi...,Open,No,No,...,NaN,2018-10-01 10:32:12,"Munsell,Julia E.",2018-10-01 10:32:12,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST,Other
3,13957,2018-10-01,NaT,"Bush, Andrew",Tech Rating,"Miletich,Denise",\n​Tech RAting error,Pending Follow-Up Outside FRC,No,No,...,INC0692683,2018-10-01 10:29:46,"Bush,Andrew",2018-10-01 10:29:46,"Bush,Andrew",201810,-,Item,ccm/FRC/Lists/TEST,Other
4,13956,2018-10-01,NaT,"Ewing, Jamie",Submission Documents in EPC,"Howard,Anne",\n​\nI'm not able to open new business submiss...,Open,No,No,...,NaN,2018-10-01 09:40:43,"Munsell,Julia E.",2018-10-01 09:40:43,"Munsell,Julia E.",201810,-,Item,ccm/FRC/Lists/TEST,System Issue


In [74]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,2))
train_posts = vectorizer.fit_transform(df_requests['Request Description'])
train_posts.shape

(8642, 149172)

In [72]:
train_size = int(len(df_requests) * .8)
train_posts = df_requests['Request Description']#[:train_size]
train_tags = df_requests['key_words']#[:train_size]
test_posts = df_requests['Request Description']#[train_size:]
test_tags = df_requests['key_words']#[train_size:]

In [77]:
train_tags.shape

(8642,)

In [17]:
vocab_size = 2000
tokenize = text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_posts)

In [73]:
train_posts.shape

(8642,)

In [18]:
x_train = tokenize.texts_to_matrix(train_posts)

In [78]:
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)
y_train.shape

(8642, 17)

In [79]:
model = Sequential()
model.add(Dense(256, input_shape=(149172,)))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

history = model.fit(train_posts, y_train, 
                    batch_size=100, 
                    epochs=2, 
                    verbose=1, 
                    validation_split=0.1)

Train on 7777 samples, validate on 865 samples
Epoch 1/20
7777/7777 [==============================] - 714s 92ms/step - loss: 1.5134 - acc: 0.6743 - val_loss: 1.1486 - val_acc: 0.7017
Epoch 2/20
7777/7777 [==============================] - 868s 112ms/step - loss: 0.2750 - acc: 0.9180 - val_loss: 1.3982 - val_acc: 0.6844
Epoch 3/20


KeyboardInterrupt: 

In [62]:
y_train.shape

(6913, 17)

In [36]:
df_list = df_requests['Key Word Roll-Up'].value_counts()

In [37]:
writer = pd.ExcelWriter('categories_output.xlsx')
df_list.to_excel(writer,'Sheet1')
writer.save()

In [25]:
for i in range(10):    
    prediction = model.predict(np.array([x_test[i]]))

NameError: name 'x_test' is not defined